<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Daily_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Installation des dépendances nécessaires
!pip install farm-haystack
!pip install torch torchvision torchaudio

# 2. Importation des bibliothèques requises
import logging
from haystack.nodes import FARMReader
from haystack.utils import clean_wiki_text

# 3. Configuration du logging
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.INFO)
logging.getLogger("haystack").setLevel(logging.INFO)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 15.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=85014d6eefa4369d370670e43896932122fb2001bf473bb076fddebcaf8c7225
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc744

In [16]:
# Création d'un dataset d'entraînement propre
context = """Solweig & Izar est un laboratoire créatif spécialisé dans l'IA pour le luxe, la beauté et le design. Ludovic Veltz est le Chief Technical Officer, spécialisé en IA générative, Machine Learning et développement Python avancé. La société collabore avec des clients comme Loewe, L'Oréal, Porsche, Teenage Engineering, et Spitfire Audio. Leur système de tagging propriétaire capture l'ADN créatif spécifique à chaque marque, permettant une analyse des marqueurs identitaires et la création de datasets personnalisés."""

# Fonction pour trouver l'index exact
def find_exact_answer_start(context, answer):
    index = context.find(answer)
    if index == -1:
        raise ValueError(f"Réponse '{answer}' non trouvée dans le contexte")
    return index

# Préparation des réponses avec vérification des indices
answers = {
    "role_veltz": "Chief Technical Officer, spécialisé en IA générative, Machine Learning et développement Python avancé",
    "clients": "Loewe, L'Oréal, Porsche, Teenage Engineering, et Spitfire Audio",
    "tagging": "capture l'ADN créatif spécifique à chaque marque, permettant une analyse des marqueurs identitaires et la création de datasets personnalisés"
}

# Création du dataset avec vérification
training_data = {
    "data": [
        {
            "title": "Solweig & Izar",
            "paragraphs": [
                {
                    "context": context,
                    "qas": [
                        {
                            "question": "Quel est le rôle de Ludovic Veltz ?",
                            "id": "1",
                            "answers": [
                                {
                                    "text": answers["role_veltz"],
                                    "answer_start": find_exact_answer_start(context, answers["role_veltz"])
                                }
                            ]
                        },
                        {
                            "question": "Avec quelles marques collabore Solweig & Izar ?",
                            "id": "2",
                            "answers": [
                                {
                                    "text": answers["clients"],
                                    "answer_start": find_exact_answer_start(context, answers["clients"])
                                }
                            ]
                        },
                        {
                            "question": "Comment fonctionne leur système de tagging ?",
                            "id": "3",
                            "answers": [
                                {
                                    "text": answers["tagging"],
                                    "answer_start": find_exact_answer_start(context, answers["tagging"])
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ]
}

# Sauvegarde du dataset propre
import json
with open('train.json', 'w', encoding='utf-8') as f:
    json.dump(training_data, f, ensure_ascii=False, indent=4)

# Réinitialisation et entraînement du modèle
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

reader.train(
    data_dir="./",
    train_filename="train.json",
    n_epochs=3,
    learning_rate=1e-5,
    max_seq_len=384,
    warmup_proportion=0.2,
    evaluate_every=100,
    save_dir="my_qa_model_improved"
)


# Test du modèle
from haystack.schema import Document

# Création du document de test
doc = Document(content=training_data["data"][0]["paragraphs"][0]["context"])

# Questions de test
questions = [
    "Quel est le rôle de Ludovic Veltz ?",
    "Avec quelles marques collabore Solweig & Izar ?",
    "Comment fonctionne leur système de tagging ?"
]

# Test du modèle
for question in questions:
    predictions = reader.predict(
        query=question,
        documents=[doc],
        top_k=1
    )
    print(f"\nQuestion: {question}")
    print(f"Réponse: {predictions['answers'][0].answer}")
    print(f"Score de confiance: {predictions['answers'][0].score:.2f}")




INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|
 (o)(o)------'\ _ /     ( )
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:Loading train set from: train


Question: Quel est le rôle de Ludovic Veltz ?
Réponse: Chief Technical Officer
Score de confiance: 0.84


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 96.09 Batches/s]



Question: Avec quelles marques collabore Solweig & Izar ?
Réponse: Loewe, L'Oréal, Porsche, Teenage Engineering, et Spitfire Audio
Score de confiance: 0.90


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 58.61 Batches/s]


Question: Comment fonctionne leur système de tagging ?
Réponse: capture l'ADN créatif spécifique à chaque marque
Score de confiance: 0.37


In [18]:
# 1. Import des composants nécessaires
from haystack.nodes import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack.pipelines import ExtractiveQAPipeline
from haystack.schema import Document

# 2. Création de notre base de documents
document_store = InMemoryDocumentStore(use_bm25=True)

# 3. Préparation des documents avec les informations de Solweig & Izar
documents = [
    Document(
        content="""Solweig & Izar est un laboratoire créatif spécialisé dans l'IA pour le luxe, la beauté et le design.
        La société est située au 4B Rue Pierre Guérin, 75016 Paris, France.""",
        meta={"type": "company_info"}
    ),
    Document(
        content="""L'équipe est dirigée par Yannis Mouhoun, Creative Director expert en esthétique des marques de luxe,
        et Ludovic Veltz, Chief Technical Officer spécialisé en IA générative et Machine Learning.""",
        meta={"type": "team"}
    ),
    Document(
        content="""Les services incluent la création de workflows génératifs, la production de contenu itératif,
        et le développement de concepts créatifs originaux. La société collabore avec des marques comme Gucci,
        Cartier, Loewe, L'Oréal et Porsche.""",
        meta={"type": "services"}
    ),
    Document(
        content="""Leur système de tagging propriétaire capture l'ADN créatif de chaque marque, permettant
        de réduire les temps de production de 6 à 2,5 mois tout en maintenant les standards du luxe.""",
        meta={"type": "technology"}
    )
]

# 4. Ajout des documents au document store
document_store.write_documents(documents)

# 5. Création du retriever
retriever = BM25Retriever(document_store=document_store)

# 6. Création du pipeline complet
pipe = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# 7. Test du pipeline avec des questions variées
test_questions = [
    "Où sont situés les bureaux de Solweig & Izar ?",
    "Quels sont les principaux clients de l'entreprise ?",
    "Comment leur technologie améliore-t-elle la production ?",
    "Quels services proposent-ils ?"
]

print("Test du pipeline complet :")
print("=" * 50)

for question in test_questions:
    prediction = pipe.run(
        query=question,
        params={
            "Retriever": {"top_k": 2},
            "Reader": {"top_k": 1}
        }
    )

    answer = prediction['answers'][0]
    print(f"\nQuestion: {question}")
    print(f"Réponse: {answer.answer}")
    print(f"Score: {answer.score:.2f}")
    print(f"Document: {answer.meta['type']}")
    print("-" * 30)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
Updating BM25 representation...: 100%|██████████| 4/4 [00:00<00:00, 30954.27 docs/s]


Test du pipeline complet :


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 58.36 Batches/s]



Question: Où sont situés les bureaux de Solweig & Izar ?
Réponse: La société est située au 4B Rue Pierre Guérin, 75016 Paris, France
Score: 0.31
Document: company_info
------------------------------


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 72.06 Batches/s]



Question: Quels sont les principaux clients de l'entreprise ?
Réponse: Cartier, Loewe, L'Oréal et Porsche
Score: 0.42
Document: services
------------------------------


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 71.44 Batches/s]



Question: Comment leur technologie améliore-t-elle la production ?
Réponse: développement de concepts créatifs originaux
Score: 0.19
Document: services
------------------------------


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 72.41 Batches/s]


Question: Quels services proposent-ils ?
Réponse: la création de workflows génératifs, la production de contenu itératif, 
        et le développement de concepts créatifs originaux
Score: 0.51
Document: services
------------------------------
